In [1]:
%reset

In [3]:
import sys
sys.path.append('/home/users/ids29/DGRB')

In [4]:
import aegis
import numpy as np
import torch
import healpy as hp
import pickle as pk
from astropy import units
from astropy import constants as c
import matplotlib.pyplot as plt
from os import listdir
import os
from sbi.inference import SNLE, SNPE#, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
# from sbi.inference.base import infer
from getdist import plots, MCSamples
from joblib import Parallel, delayed
from scipy.integrate import quad, simpson
import pickle
from scipy.stats import norm
import sources.DMsignal as DMsignal
import sources.FermiBackgrounds as FermiBackgrounds


%matplotlib inline

In [4]:
grains=1000
num_simulations = 1000
num_workers = 48

In [ ]:
parameter_range_aegis = [[], []]
abundance_luminosity_and_spectrum_list = []
source_class_list = []
parameter_names = []
energy_range = [1000, 100000] #MeV 
energy_range_gen = [energy_range[0]*0.5, energy_range[1]*1.5]
# max_radius = 8.5 + 20*2 #kpc
max_radius = 220 #kpc #This is the virial radius of the Milky Way
exposure = 2000*10*0.2 #cm^2 yr
flux_cut = 1e-9 #photons/cm^2/s
angular_cut = 180*units.deg.to('rad') #degrees #np.pi 
angular_cut_gen =  min(angular_cut*1.5, np.pi) # np.pi
lat_cut = 30 #2*u.deg.to('rad') #degrees
lat_cut_gen = lat_cut*0.5

In [6]:
my_cosmology = 'Planck18'
# z_range = [0, 14]
luminosity_range = 10.0**np.array([33, 50]) # Minimum value set by considering source at distnace of closest approach by line of sight at 40 degrees and receiving 0.1 photon at detector side. CHANGE THIS FOR FINAL PROBLEM.
my_AEGIS = aegis.aegis(abundance_luminosity_and_spectrum_list, source_class_list, parameter_range_aegis, energy_range, luminosity_range, max_radius, exposure, angular_cut, lat_cut, flux_cut, energy_range_gen=energy_range_gen, cosmology = my_cosmology, verbose = False)
my_AEGIS.angular_cut_gen, my_AEGIS.lat_cut_gen = angular_cut_gen, lat_cut_gen

In [7]:
# a simple simulator with the total number of photons as the summary statistic
def simulator(params):

    input_params = params.numpy()
    print(f"Input parameters: {input_params}")

    source_info = my_AEGIS.create_sources(input_params, grains=grains, epsilon=1e-2)
    photon_info = my_AEGIS.generate_photons_from_sources(input_params, source_info, grains=grains) 
    obs_info = {'psf_fits_path': '/home/users/ids29/DGRB/FERMI_files/psf_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'edisp_fits_path': '/home/users/ids29/DGRB/FERMI_files/edisp_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'event_type': 'PSF3', 'exposure_map': None}
    obs_photon_info = my_AEGIS.mock_observe(photon_info, obs_info)
    
    return obs_photon_info

Isotropic Background signal

In [8]:
# # Add Fermi isotropic background source class
# data_root = '/home/users/ids29/DGRB'
# my_FB = FermiBackgrounds.FermiBackgrounds(data_root)
# def spec_iso_wrap(energy, params):
#     A_BG = params[0]
#     iso_fit = my_FB.get_isotropic_background_spectrum_func()
#     return A_BG * iso_fit(energy)

In [9]:
# als_FIB = [spec_iso_wrap]
# my_AEGIS.abun_lum_spec = [als_FIB] # Change when you want to include Poisson model.
# my_AEGIS.source_class_list = ['isotropic_diffuse']

In [10]:
# test_params = torch.tensor([1])
# test_photon_info = simulator(test_params)
# print(f"Number of photons = {test_photon_info['energies'].size}")
# # Input parameters: [1]
# # Number of photons = 351854

Non-isotropic Background signal

In [11]:
data_root = '/home/users/ids29/DGRB'
my_FB      = FermiBackgrounds.FermiBackgrounds(data_root)

def spec_non_iso_wrap(params):
    """
    Returns (vals, E_edges, pix_idx, nside) in the format AEGIS expects
    for a 'healpix_map' source.  The `params` argument is ignored – it is
    only present because AEGIS will pass the global parameter vector in.
    """
    nside        = 64                     # keep it power-of-2 (same as AEGIS default)
    ang_cut      = my_AEGIS.angular_cut_gen   # reuse whatever ROI you set earlier
    Emin, Emax   = energy_range_gen[0], energy_range_gen[1]
    N_Ebins      = 10                     # or any number of energy slices you like

    vals, E_edges, pix_idx = my_FB.get_partial_nonistropic_background(
                                angular_cut = ang_cut,
                                Emin        = Emin,
                                Emax        = Emax,
                                N_Ebins     = N_Ebins,
                                N_side      = nside)
    return vals, E_edges, pix_idx, nside

In [12]:
als_non_iso_background = [spec_non_iso_wrap]
my_AEGIS.abun_lum_spec = [als_non_iso_background] # Change when you want to include Poisson model.
my_AEGIS.source_class_list = ['healpix_map']

In [ ]:
test_params = torch.tensor([1])  # No parameters needed for healpix_map
test_photon_info = simulator(test_params)
print(f"Number of photons = {test_photon_info['energies'].size}")
# Input parameters: [1]
# Healpix map 0 has 540672 pixels
# Number of photons = 217035

Input parameters: [1]
Healpix map 0 has 540672 pixels
Number of photons = 217035
